In [1]:
import os
import json
import re

# 데이터 전처리 함수 정의
def clean_text(text):
    """
    텍스트에서 불필요한 공백 및 특수 문자를 제거합니다.
    """
    text = re.sub(r"\s+", " ", text)  # 다중 공백을 하나로 변환
    text = text.strip()  # 양 끝 공백 제거
    return text

def extract_characters_and_lines(passage):
    """
    등장인물과 대화를 추출합니다.
    """
    lines = passage.split("\n")
    dialogues = []
    for line in lines:
        match = re.match(r"(.*?)\](.*)", line)  # 예: 기준모] 대사
        if match:
            character, dialogue = match.groups()
            dialogues.append({"character": character.strip(), "dialogue": clean_text(dialogue)})
    return dialogues

def preprocess_data(data):
    """
    JSON 데이터셋을 전처리하여 필요한 정보만 추출합니다.
    """
    meta = data.get("Meta", {})
    annotation = data.get("Annotation", {})

    # 주요 정보 추출
    passage = meta.get("passage", "")
    summaries = {key: clean_text(value) for key, value in annotation.items() if value}

    # 등장인물 및 대사 추출
    dialogues = extract_characters_and_lines(passage)

    # 전처리된 결과 반환
    return {
        "doc_id": meta.get("doc_id"),
        "passage": dialogues,
        "summaries": summaries
    }

def create_preprocessed_dir_structure(base_dir):
    """
    기존 디렉토리 구조를 바탕으로 'pre_' 접두사를 붙인 새로운 디렉토리 구조 생성
    """
    for root, dirs, files in os.walk(base_dir):
        for dir_name in dirs:
            new_dir = os.path.join(root, f"pre_{dir_name}")
            if not os.path.exists(new_dir):
                os.makedirs(new_dir)

def process_all_json_files(base_dir):
    """
    모든 JSON 파일을 순회하며 전처리하고 새로운 폴더에 저장
    """
    for root, dirs, files in os.walk(base_dir):
        if not any(d.startswith("pre_") for d in dirs):
            # 새로운 디렉토리 생성
            create_preprocessed_dir_structure(root)

        for file in files:
            if file.endswith(".json"):
                file_path = os.path.join(root, file)

                # 파일 읽기
                with open(file_path, "r", encoding="utf-8") as f:
                    data = json.load(f)

                # 데이터 전처리
                processed_data = preprocess_data(data)

                # 저장 경로 생성
                relative_path = os.path.relpath(root, base_dir)
                preprocessed_dir = os.path.join(base_dir, f"pre_{relative_path}")
                preprocessed_file_path = os.path.join(preprocessed_dir, file)

                # 파일 저장
                os.makedirs(preprocessed_dir, exist_ok=True)
                with open(preprocessed_file_path, "w", encoding="utf-8") as f:
                    json.dump(processed_data, f, ensure_ascii=False, indent=4)

# 실행 코드
base_directory = "./"  # 루트 디렉토리 경로 설정
process_all_json_files(base_directory)


In [1]:
import json
import re

# 데이터 전처리 함수 정의
def clean_text(text):
    """
    텍스트에서 불필요한 공백 및 특수 문자를 제거합니다.
    """
    text = re.sub(r"\\s+", " ", text)  # 다중 공백을 하나로 변환
    text = text.strip()  # 양 끝 공백 제거
    return text

def extract_characters_and_lines(passage):
    """
    등장인물과 대화를 추출합니다.
    """
    lines = passage.split("\\n")
    dialogues = []
    for line in lines:
        match = re.match(r"(.*?)\\](.*)", line)  # 예: 기준모] 대사
        if match:
            character, dialogue = match.groups()
            dialogues.append({"character": character.strip(), "dialogue": clean_text(dialogue)})
    return dialogues

def preprocess_data(data):
    """
    JSON 데이터셋을 전처리하여 필요한 정보만 추출합니다.
    """
    meta = data.get("Meta", {})
    annotation = data.get("Annotation", {})

    # 주요 정보 추출
    passage = meta.get("passage", "")
    summaries = {key: clean_text(value) for key, value in annotation.items() if value}

    # 등장인물 및 대사 추출
    dialogues = extract_characters_and_lines(passage)

    # 전처리된 결과 반환
    return {
        "doc_id": meta.get("doc_id"),
        "passage": dialogues,
        "summaries": summaries
    }

# JSON 파일 로드
def load_json_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)
    return data

# 데이터 파일 경로
file_path = "culture/3sent/SCRIPT-culture-00001-00006.json"

# 데이터 로드 및 전처리
raw_data = load_json_file(file_path)
processed_data = preprocess_data(raw_data)

# 전처리된 데이터 저장 (옵션)
output_file = "processed_data.json"
with open(output_file, "w", encoding="utf-8") as file:
    json.dump(processed_data, file, ensure_ascii=False, indent=4)

# 결과 출력
print(json.dumps(processed_data, ensure_ascii=False, indent=4))


{
    "doc_id": "SCRIPT-culture-00001",
    "passage": [],
    "summaries": {
        "Summary1": "경기가 끝나고 해변가에서 열린 파티에 참석했고, 다음날 시장의 초대로 작은 마을에 가서 아사도를 먹었다.",
        "Summary2": "경기가 끝나고 폴로 선수와 마을 사람들까지 모여 해변에서는 파티가 열렸다. 200인분의 파에야가 완성되고, 촬영하는 취재진까지 챙겨주신 '아저씨' 덕에 발데스 반도에서의 하루가 파티로 마무리되었다. 다음날 시장의 초대로 베인 티오 초 데 홀리오라는 작은 마을에 가서 그곳 사람들이 평소 주말에 먹는다는 아사도를 먹었다."
    }
}
